In [ ]:
import json
import pandas as pd
from pathlib import Path
from pandas import json_normalize
from tqdm import tqdm

In [ ]:
assessment_data = Path(f'out/assessment_data')
assessment_data.exists()

In [ ]:
assessment_dir_paths = list(assessment_data.glob('*/'))
assessment_dir_paths

In [ ]:
def get_dswx_ids(assessment_dir) -> list:
    dswx_verification_paths = list(assessment_dir.glob('*/'))
    dswx_ids = [path.name for path in dswx_verification_paths]
    # Remove DSWx paths
    dswx_ids = list(filter(lambda dswx_id: '.' != dswx_id[0], dswx_ids))
    return dswx_ids


def combine_requirement_verification_for_one_assessment(assessment_dir_path):
    out_dir = Path(f'out/verification_stats_agg/{assessment_dir_path.name}')
    out_dir.mkdir(exist_ok=True, parents=True)
    
    
    dswx_ids = get_dswx_ids(assessment_dir_path)
    
    def read_one_requirements_json(dswx_id):
        path = assessment_dir_path / dswx_id / f'requirement_verification_{dswx_id}.json'
        data = json.load(open(path))
        return data
    
    data = list(map(read_one_requirements_json, dswx_ids))
    df = pd.DataFrame(data)
    
    cols = ['surface_water','partial_surface_water']
    df[cols + ['dswx_id']].groupby(cols).count()
    
    out_path = out_dir / 'verification_results.csv'
    df.to_csv(out_path, index=False)
    
    return df, out_path

In [ ]:
dfs, out_paths = zip(*list(map(combine_requirement_verification_for_one_assessment, tqdm(assessment_dir_paths))))

In [ ]:
dfs[0]

In [ ]:
out_paths[0]

# Additional Metrics

In [ ]:
def combine_metrics_for_one_assessment(assessment_dir_path):
    out_dir = Path(f'out/verification_stats_agg/{assessment_dir_path.name}')
    
    def read_one_assessment(dswx_id):
        path = assessment_dir_path / dswx_id / f'stats_{dswx_id}.json'
        data = pd.DataFrame(pd.json_normalize(json.load(open(path))))
        return data
    
    dswx_ids = get_dswx_ids(assessment_dir_path)
    
    data_assessments = list(map(read_one_assessment, dswx_ids))
    df_metrics = pd.concat(data_assessments, axis=0).reset_index(drop=True)
    df_metrics.to_csv(out_dir / 'metrics.csv', index=False)
    out_path = out_dir / 'metrics.csv'
    df_metrics.to_csv(out_path, index=False)
    return df_metrics, out_path

In [ ]:
dfs, out_paths = zip(*list(map(combine_metrics_for_one_assessment, tqdm(assessment_dir_paths))))

In [ ]:
dfs[0]

In [ ]:
out_paths[0]